# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [2]:
!pip install --upgrade --upgrade-strategy eager azureml-sdk

Requirement already up-to-date: azureml-sdk in /anaconda/envs/azureml_py36/lib/python3.6/site-packages (1.20.0)
Requirement already up-to-date: azureml-dataset-runtime[fuse]~=1.20.0 in /anaconda/envs/azureml_py36/lib/python3.6/site-packages (from azureml-sdk) (1.20.0)
Requirement already up-to-date: azureml-train~=1.20.0 in /anaconda/envs/azureml_py36/lib/python3.6/site-packages (from azureml-sdk) (1.20.0)
Requirement already up-to-date: azureml-pipeline~=1.20.0 in /anaconda/envs/azureml_py36/lib/python3.6/site-packages (from azureml-sdk) (1.20.0)
Requirement already up-to-date: azureml-train-automl-client~=1.20.0 in /anaconda/envs/azureml_py36/lib/python3.6/site-packages (from azureml-sdk) (1.20.0)
Requirement already up-to-date: azureml-core~=1.20.0 in /anaconda/envs/azureml_py36/lib/python3.6/site-packages (from azureml-sdk) (1.20.0)
Requirement already up-to-date: azureml-dataprep<2.8.0a,>=2.7.0a in /anaconda/envs/azureml_py36/lib/python3.6/site-packages (from azureml-dataset-runti

In [3]:
!pip install --upgrade azureml-sdk[automl]

Requirement already up-to-date: azureml-sdk[automl] in /anaconda/envs/azureml_py36/lib/python3.6/site-packages (1.20.0)


In [3]:
import azureml.core
print(azureml.core.VERSION)

1.19.0


In [2]:
from azureml.core import Workspace, Experiment

ws = Workspace.get(name="quick-starts-ws-135226")
exp = Experiment(workspace=ws, name="quick-starts-ws-135226")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: quick-starts-ws-135226
Azure region: southcentralus
Subscription id: b968fb36-f06a-4c76-a15f-afab68ae7667
Resource group: aml-quickstarts-135226


In [4]:
from azureml.core.compute import ComputeTarget, AmlCompute

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.
compute_target_name = "vpcompute"
compute_cluster = AmlCompute.provisioning_configuration(vm_size = "Standard_D2_V2", max_nodes = 4)
my_target =  ComputeTarget.create(ws, compute_target_name, compute_cluster)
### YOUR CODE HERE ###

In [5]:
import logging
import os
import csv

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from sklearn import datasets
import pkg_resources

import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.train.automl import AutoMLConfig
from azureml.core.dataset import Dataset

from azureml.pipeline.steps import AutoMLStep



# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

SDK version: 1.19.0


In [7]:
#importing the libraries that we use
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
#import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
#from imblearn.over_sampling import SMOTE
#from imblearn.under_sampling import RandomUnderSampler
from sklearn.neighbors import KNeighborsClassifier
#from sklearn.naive_bayes import GaussianNB
#from sklearn.svm import SVC
#sns.set(color_codes=True) # adds a nice background to the graphs
%matplotlib inline

## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [8]:
from azureml.core.dataset import Dataset
import pandas as pd

dataset = Dataset.get_by_name(ws, name = 'Personal_loan')
P_loan = dataset.to_pandas_dataframe()
P_loan

,ID,Age,Experience,Income,ZIP Code,Family,CCAvg,Education,Mortgage,Personal Loan,Securities Account,CD Account,Online,CreditCard
0,1,25,1,49,91107,4,1.6,1,0,0,1,0,0,0
1,2,45,19,34,90089,3,1.5,1,0,0,1,0,0,0
2,3,39,15,11,94720,1,1.0,1,0,0,0,0,0,0
3,4,35,9,100,94112,1,2.7,2,0,0,0,0,0,0
4,5,35,8,45,91330,4,1.0,2,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,4996,29,3,40,92697,1,1.9,3,0,0,0,0,1,0
4996,4997,30,4,15,92037,4,0.4,1,85,0,0,0,1,0
4997,4998,63,39,24,93023,2,0.3,3,0,0,0,0,0,0
4998,4999,65,40,49,90034,3,0.5,2,0,0,0,0,1,0


In [9]:
avg_Experience = P_loan["Experience"].mean()
print(f"Average Experience {avg_Experience}")
P_loan["Experience"] = P_loan["Experience"].apply(lambda x : avg_Experience if x<0 else x)

Average Experience 20.1046


In [10]:
q1 = P_loan["Income"].quantile(0.25)
q3 = P_loan["Income"].quantile(0.75)
iqr = q3 - q1
border = q1 + 1.5 * iqr
P_inc = P_loan["Income"].apply(lambda x : border if x>border else x)
P_inc

q1 = P_loan["Mortgage"].quantile(0.25)
q3 = P_loan["Mortgage"].quantile(0.75)
iqr = q3 - q1
border = q1 + 1.5 * iqr
P_mortgage = P_loan["Mortgage"].apply(lambda x : border if x>border else x)
P_mortgage

0        0.0
1        0.0
2        0.0
3        0.0
4        0.0
        ... 
4995     0.0
4996    85.0
4997     0.0
4998     0.0
4999     0.0
Name: Mortgage, Length: 5000, dtype: float64

In [11]:
P_loan["Income"]=P_inc
P_loan["Mortgage"]=P_mortgage
P_loan.head()

,ID,Age,Experience,Income,ZIP Code,Family,CCAvg,Education,Mortgage,Personal Loan,Securities Account,CD Account,Online,CreditCard
0,1,25,1.0,49.0,91107,4,1.6,1,0.0,0,1,0,0,0
1,2,45,19.0,34.0,90089,3,1.5,1,0.0,0,1,0,0,0
2,3,39,15.0,11.0,94720,1,1.0,1,0.0,0,0,0,0,0
3,4,35,9.0,100.0,94112,1,2.7,2,0.0,0,0,0,0,0
4,5,35,8.0,45.0,91330,4,1.0,2,0.0,0,0,0,0,1


In [12]:
P_loan["Mortgage"]= np.log1p(P_loan["Mortgage"])

In [13]:
scalar = StandardScaler()
P_loan["Experience"]=scalar.fit_transform(P_loan[["Experience"]])
P_loan["Income"]=scalar.fit_transform(P_loan[["Income"]])
P_loan["CCAvg"]=scalar.fit_transform(P_loan[["CCAvg"]])
P_loan["Mortgage"]=scalar.fit_transform(P_loan[["Mortgage"]])

In [14]:
ws = Workspace.from_config()

# choose a name for experiment
experiment_name = 'Personal_loan_experiment'

experiment=Experiment(ws, experiment_name)

In [15]:
import pandas as pd
from scipy import sparse
from sklearn.model_selection import train_test_split
from sklearn import datasets
from azureml.core.dataset import Dataset
import numpy as np

#X = P_loan.drop(['Personal Loan'],axis=1)     # Predictor feature columns 

Target = P_loan["Personal Loan"]
#Y = Target   # Predicted class (1, 0) 

X = P_loan.drop(["Personal Loan","Age","ZIP Code","CreditCard","ID","Online"],axis=1)     # Predictor feature columns 
Y = Target   # Predicted class (1, 0) 

x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=1)
# 1 is just any random seed number

x_train.head()

,Experience,Income,Family,CCAvg,Education,Mortgage,Securities Account,CD Account
1334,0.148536,-0.919844,2,-0.365061,1,-0.665573,0,0
4768,-0.562456,-0.811719,1,0.035515,2,-0.665573,0,0
65,1.303898,1.580547,1,1.065567,1,-0.665573,0,0
177,-1.540070,-0.108906,4,-0.078935,2,1.571955,0,0
4489,-0.651330,-1.298281,3,-0.994537,2,-0.665573,0,0


In [16]:
def logistReg(x_train,y_train,solver="liblinear"):
    # Fit the model on train
    model = LogisticRegression(solver=solver)
    model.fit(x_train, y_train)
    #predict on test
    y_predict = model.predict(x_test)
    y_predictprob = model.predict_proba(x_test)

    coef_df = pd.DataFrame(model.coef_,columns=list(x_train.columns))
    coef_df['intercept'] = model.intercept_
    model_score = model.score(x_train, y_train)
    print(f"Accuracy of Training Data: {model_score}")
    model_score = model.score(x_test, y_test)
    print(f"Accuracy of Test Data: {model_score}")
    print(coef_df)
    print(metrics.classification_report(y_test,y_predict))
    return y_predictprob,y_predict

In [17]:
y_predProb,y_pred = logistReg(x_train,y_train)

Accuracy of Training Data: 0.9608571428571429
Accuracy of Test Data: 0.9573333333333334
   Experience    Income    Family    CCAvg  Education  Mortgage  \
0   -0.006157  3.334318  0.574265  0.23325    1.37247  0.121981   

   Securities Account  CD Account  intercept  
0           -0.717196    2.162063  -9.369101  
              precision    recall  f1-score   support

           0       0.96      0.99      0.98      1351
           1       0.90      0.64      0.75       149

    accuracy                           0.96      1500
   macro avg       0.93      0.82      0.86      1500
weighted avg       0.96      0.96      0.95      1500



In [18]:
import pandas as pd
from scipy import sparse
from sklearn.model_selection import train_test_split
from sklearn import datasets
from azureml.core.dataset import Dataset
import numpy as np
#from train import clean_data
#from azureml.data.dataset_factory import TabularDatasetFactory
from sklearn import preprocessing
import os


#x_train, x_test, y_train, y_test = train_test_split(P_loan, test_size = 0.3, random_state = 0)
#x_train, x_test, y_train, y_test
training_data = pd.concat([x_train, y_train], axis = 1)

if not os.path.isdir('data'):
    os.mkdir('data')


training_data.to_csv('data/loan.csv')
ds = ws.get_default_datastore()
ds.upload(src_dir='./data', target_path='loandata', overwrite=True, show_progress=True)

training_data = Dataset.Tabular.from_delimited_files(path=ds.path('loandata/loan.csv'))
training_data

Uploading an estimated of 1 files
Uploading ./data/loan.csv
Uploaded ./data/loan.csv, 1 files out of an estimated total of 1
Uploaded 1 files


{
  "source": [
    "('workspaceblobstore', 'loandata/loan.csv')"
  ],
  "definition": [
    "GetDatastoreFiles",
    "ParseDelimited",
    "DropColumns",
    "SetColumnTypes"
  ]
}

## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [19]:
# TODO: Put your automl settings here
automl_settings = {
     "experiment_timeout_hours" : 0.3,
    "enable_early_stopping" : True,
    "iteration_timeout_minutes": 5,
    "max_concurrent_iterations": 4,
    "primary_metric": 'AUC_weighted',
    "featurization": 'auto',
}

# TODO: Put your automl config here
automl_config = AutoMLConfig(
    compute_target = 'vpcompute',
    task="classification",
    training_data= training_data,
    label_column_name= "Personal Loan",
    n_cross_validations=5
)

In [20]:
# TODO: Submit your experiment
#remote_run = experiment.submit(automl_config)
remote_run = experiment.submit(automl_config, show_output = True)
remote_run

Running on remote.
No run_configuration provided, running on vpcompute with default configuration
Running on remote compute: vpcompute
Parent Run ID: AutoML_04a76343-05bc-433f-b65c-cbe126eaf4d6

Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.
Current status: ModelSelection. Beginning model selection.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       ALERTED
DESCRIPTION:  To decrease model bias, please cancel the current run and fix balancing problem.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData
DETAILS:      Imbalanced data can lead to a falsely perceived positive effect of a model's accuracy because the input data has bias towards one class.
+---------------------------------+-----------------------------

Experiment,Id,Type,Status,Details Page,Docs Page
Personal_loan_experiment,AutoML_04a76343-05bc-433f-b65c-cbe126eaf4d6,automl,Completed,Link to Azure Machine Learning studio,Link to Documentation


## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [21]:
from azureml.widgets import RunDetails

RunDetails(remote_run).show()

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [22]:
best_run, best_model = remote_run.get_output()

In [23]:
#TODO: Save the best model
best_run.register_model(model_name = 'ml_best_model.pkl', model_path = './outputs/')

Model(workspace=Workspace.create(name='quick-starts-ws-135226', subscription_id='b968fb36-f06a-4c76-a15f-afab68ae7667', resource_group='aml-quickstarts-135226'), name=ml_best_model.pkl, id=ml_best_model.pkl:1, version=1, tags={}, properties={})

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

TODO: In the cell below, send a request to the web service you deployed to test it.

TODO: In the cell below, print the logs of the web service and delete the service